In [1]:
import shutil
import os
from skimage.exposure import match_histograms

%run ./variables.ipynb
%run ./utils.ipynb
%run ../utils/data_utils.ipynb
%run ../utils/image_utils.ipynb

# Loading reference image for histogram matching
ref = cv2.imread("/mnt/nvme-storage/pfauregi/datasets/atlas/ref_img.png")
ref = cv2.cvtColor(ref, cv2.COLOR_BGR2GRAY)

In [2]:
selected_taxons = get_selected_taxons(SELECTED_TAXONS)
for path in ATLAS_PATH:
    print(path)
    for taxon in os.listdir(path):
        if taxon in selected_taxons.keys():
            dir_path = os.path.join(path, taxon)
            files = [f for f in os.listdir(dir_path) if isfile(join(dir_path, f))]
            for file in files:
                split = file.split(".")
                if (len(split)>1 and split[1]=="png"):
                    source_file = os.path.join(dir_path, file)
                    target_file = os.path.join(DATASET_PATH, taxon, file)
                    check_dirs(target_file)
                    img = cv2.imread(source_file)
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                    img = match_histograms(img, ref, multichannel=False)
                    img = convert_to_square(img, new_size=256)
                    cv2.imwrite(target_file, img)

/mnt/nvme-storage/pfauregi/datasets/atlas/BRG
/mnt/nvme-storage/pfauregi/datasets/atlas/IDF


In [ ]:
from IPython.display import display
from PIL import Image
from skimage import data
from skimage import exposure

id_map = get_selected_taxons(SELECTED_TAXONS)
X, y, _ = get_dataset(ATLAS_PATH[0], ids=False)

In [ ]:
ref = cv2.imread("/mnt/nvme-storage/pfauregi/datasets/atlas/ref_img.png")
ref = cv2.cvtColor(ref, cv2.COLOR_BGR2GRAY)
src = cv2.imread(X[0])
src = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)

corrected_src = match_histograms(src, ref, multichannel=False)


display(Image.fromarray(src))
display(Image.fromarray(ref))
display(Image.fromarray(corrected_src.astype('uint8')))